In [224]:
from __future__ import division
import scipy
import binascii
from sklearn.metrics import f1_score
import numpy as np
import base64
from sklearn.linear_model import LogisticRegression
import csv
from nltk.tokenize import RegexpTokenizer
import gzip
import pymorphy2
import zlib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import importlib
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import xgboost
import zipfile
import gzip
import string
import pickle
import re
import sys
from io import BytesIO
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from collections import namedtuple
from scipy import sparse
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
%matplotlib inline
from bs4 import BeautifulSoup
from string import punctuation
from bs4.element import Comment
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.tree import DecisionTreeClassifier  
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from zipfile import ZipFile 
import gzip
import shutil
import re
import operator
from functools import reduce
import pandas as pd
import ast
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /home/linuxconfig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# небольшой пресказ хода работы
# сначала просто в лоб парсим все сайты и получаем индекс который  представляет из себя словарь где ключ это id страницы, а значение это список где 1ый элемент это урл а второй это слова на  странице(уже привеенные в норм форму)
# далее я создал лист всех текстов страниц обьединил его и сделалсет(словарь) всех слов которые есть 
# далее пробежался по этому сету и отскал сколько раз и в каких документах встречается слово (сделал в лоб) может показаться что это тупо - да так и есть но доков не много и все было довольно быстро (заварил чай поситал хабр и готово)
# далее идет обработка запроса и выдача 
# еще говорили что надо использовать сжатие индекса я его реализовал но не внедрил это в модель (внедрить не сложно просто пробегаемся по обратному индексу и шифруем постинг листы, когда надо достать инфу то дешифруем данные и берем)
# не иплементировал сжатие так как simple9 хорош когда постинг листы очень длинный но тут постинг листы довольно короткие
# если выдача будет не очень точной скорее вссего при приведении в норм форму пайморфи мог немного ошибиться 

In [ ]:
pages = []
morph = morph = pymorphy2.MorphAnalyzer()
names = ['data_bool/lenta.ru_4deb864d-3c46-45e6-85f4-a7ff7544a3fb_01.gz',
         'data_bool/lenta.ru_80e74243-83da-4367-8ae3-fe38d333f283_01.gz',
         'data_bool/lenta.ru_159b9f4b-972b-48b1-8ec3-44fbd6be33c4_01.gz',
         'data_bool/lenta.ru_6398c7e2-16da-40d2-8923-95f65aaaeb07_01.gz',
         'data_bool/lenta.ru_aa5a1ef9-6ca4-4dc7-890f-308d4d62db59_01.gz',
         'data_bool/lenta.ru_b81aa623-ba55-43dc-b3c5-47ae2253ad27_01.gz',
         'data_bool/lenta.ru_b6838708-1aa9-496f-bf88-e277374f93a8_01.gz',
         'data_bool/lenta.ru_d1f7e910-b5f1-4719-b724-090093e143fe_01.gz']
for i in names:
    file = gzip.open('data_bool/lenta.ru_4deb864d-3c46-45e6-85f4-a7ff7544a3fb_01.gz')
    content = file.read()
    pages.append(content)
separate_pages = []
for i in pages:
    separate_pages.append(re.split(r'http:',i.decode("utf-8",'ignore')))
all_pages_sep = reduce(operator.concat, separate_pages)
all_pages_sep = ['http:'+item for item in all_pages_sep if len(item)>101]
doc_id = 0
index = {}
regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
for i in all_pages_sep:
    i = i.lower()
    url = re.findall(regex,i)[0][0]
    words = re.findall(r'\w+',i)
    words = [morph.parse(j)[0].normal_form for j in words]
    index[int(doc_id)] = [url,words]
    doc_id +=1
word_corpus_sep = np.array(list(index.values()))[:,1:]

In [213]:
# with open('index.pickle', 'wb') as handle:
#     pickle.dump(index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('index.pickle', 'rb') as handle:
    index = pickle.load(handle)

# print(index == index_r)

In [43]:
word_corpus_sep = np.array(list(index.values()))[:,1:]
word_corpus = []
for i in word_corpus_sep:
    for j in i:
        word_corpus.append(j)
w = []
for i in word_corpus_sep:
    for j in i:
        for j in j:
            w.append(j)
words_corpus_set = set(w)

In [54]:
reverse_index = {}
doc_id = 0
for i in words_corpus_set:
    imbed = []
    doc_id = 0
    for j in word_corpus:
        im = j.count(i)
        if im == 0:
            doc_id += 1
        else:
            imbed.append([doc_id,im])
        
    reverse_index[i] = imbed

In [55]:
with open('reverse_index.pickle', 'wb') as handle:
    pickle.dump(reverse_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('reverse_index.pickle', 'rb') as handle:
    reverse_index_r = pickle.load(handle)

print(reverse_index == reverse_index_r)

True


In [ ]:
en_simple = [[28,         1, SIMPLE_9, 1],
             [14,         3, SIMPLE_8, 2],
             [ 9,         7, SIMPLE_7, 3],
             [ 7,        15, SIMPLE_6, 4],
             [ 5,        31, SIMPLE_5, 5],
             [ 4,       127, SIMPLE_4, 7],
             [ 3,       511, SIMPLE_3, 9],
             [ 2,     16383, SIMPLE_2,14],
             [ 1, 268435455, SIMPLE_1,28]
           ]
def simple9_encode(a, length):
    global en_simple
    off = 0        # count encode length
    result = []
    while off < length:
        for t in en_simple:
            n, threshold, code, shift = t[0], t[1], t[2], t[3]
            if off + n <= length and max(a[off:off + n]) <= t[1]:
                tmp = a[off]
                for i in range(1, n):
                    tmp |= (a[off + i] << (shift * i))
                result.append(tmp | t[2])
                off += n
                break
    return result
def simple9_decode(a):
    de_simple = {SIMPLE_9: [28, 0x1, 1],
                 SIMPLE_8: [14, 0x3, 2],
                 SIMPLE_7: [9, 0x7, 3],
                 SIMPLE_6: [7, 0xf, 4],
                 SIMPLE_5: [5, 0x1f, 5],
                 SIMPLE_4: [4, 0x7f, 7],
                 SIMPLE_3: [3, 0x1ff, 9],
                 SIMPLE_2: [2, 0x3fff, 14],
                 SIMPLE_1: [1, 0xfffffff, 28]
                 }
    result = []
    for t in a:
        code = t & 0xf0000000
        data = t & 0xfffffff
        info = de_simple[code]
        n, bit, shift = info[0], info[1], info[2]
        for i in range(n):
            result.append(data & bit)
            data >>= shift
    return result

In [269]:
def boolean_query(string, rec_level=0):
    first_arg = None
    second_arg = None
    morph = pymorphy2.MorphAnalyzer()
    if rec_level == 0 and '&' not in string and '|' not in string:
        first_arg = morph.parse(string)[0].normal_form
        list1 = np.array(reverse_index[first_arg])[:,0]
        return list1
    if '(' in string:
        possible_arg = string.split('(')[0]
        if len(possible_arg) > 2:
            first_arg = possible_arg.split('&')[0].split('|')[0]
            first_arg = morph.parse(first_arg)[0].normal_form
            list1 = np.array(reverse_index[first_arg])[:,0]
            if string[len(first_arg)] == '&':
                list2 = boolean_query(string[(len(first_arg) + 2):-1], rec_level = rec_level + 1)
                total_list = np.sort(np.intersect1d(list1, list2))
            else:
                list2 = boolean_query(string[(len(first_arg) + 2):-1], rec_level = rec_level + 1)
                total_list = np.sort(np.append(list1, list2))
            return total_list
    else:
        first_arg = string.split('&')[0].split('|')[0]
        first_arg = morph.parse(first_arg)[0].normal_form
        if string[len(first_arg)] == '&':
            second_arg = string.split('&')[1]
            second_arg = morph.parse(second_arg)[0].normal_form
            list1 =np.array(reverse_index[first_arg])[:,0]
            list2 = np.array(reverse_index[second_arg])[:,0]
            total_list =  np.sort(np.intersect1d(list1, list2))
            print(len(total_list))
            return total_list
        else:
            second_arg = string.split('|')[1]
            second_arg = morph.parse(second_arg)[0].normal_form
            list1 = np.array(reverse_index[first_arg])[:,0]
            list2 = np.array(reverse_index[second_arg])[:,0]
            total_list = np.sort(np.append(list1, list2))
            return total_list

In [284]:
def simple_snippet(query):
    a =  boolean_query(query)
    for i in a:
        url = '/'.join(np.array(index[i])[0].split('/')[0:-1])
        print(i,url)

In [293]:
query = input()
simple_snippet(query)

пожар&беда
1
5174 http://lenta.ru/news/2011/02/09/tweet
